In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from fdua_competition.vectorstore import build_vectorstore
from fdua_competition.enums import Mode, VectorStoreOption
from fdua_competition.rag import ResearchAssistant, RAG
from pprint import pprint

In [ ]:
vectorstore = build_vectorstore("one_assistant_test", Mode.TEST, VectorStoreOption.CHROMA)

In [ ]:
assistant = ResearchAssistant(vectorstore)

In [ ]:
res = assistant.invoke("電通についてまとめて")
print(res)

In [ ]:
# RAGクラスを継承して、新しいRAGを作成する例

## レスポンスの型式を定義
class FactCheckerResponse(BaseModel):
    score: str = Field(description="score")
    reason: str = Field(description="reason")


# RAGクラスを継承して新しいRAGクラスを定義
# prompt_template(), build_payload(), output_structure() メソッドをオーバーライドする
class FactChecker(RAG):
    def __init__(
        self, vectorstore: VectorStore, chat_model_option: ChatModelOption = ChatModelOption.AZURE, language: str = "japanese"
    ):
        super().__init__(vectorstore=vectorstore, chat_model_option=chat_model_option)
        self.language = language

    @property
    def prompt_template(self) -> ChatPromptTemplate:
        """プロンプトのテンプレート"""
        return ChatPromptTemplate.from_messages(
            [
                ("system", "fact check given ai response"),
                ("system", "context:\n{context}"),
                ("user", "ai-response:\n{ai_response}"),
            ]
        )

    def build_payload(self, query: str) -> dict[str, t.Any]:
        """テンプレートこの辞書で埋める"""
        return {
            "ai_response": query,
            "language": self.language,
        }

    @property
    def output_structure(self) -> BaseModel:
        """出力型式を定義する"""
        return FactCheckerResponse


# [end: research_assistant]

In [ ]:
fact_checker = FactChecker(vectorstore)
fact_checker.invoke(f"{assitant_res.query}: {assitant_res.response}")